In [4]:
from pickle import FALSE
from flask import Flask
from selenium import webdriver
import time
from datetime import datetime
import json
import pandas as pd
import os

In [2]:
def toJson(dir, dict):
    with open(dir, 'w', encoding='utf-8') as file:
        json.dump(dict, file, ensure_ascii=False, indent='\t')

def loadJson(dir):
    with open(dir, 'r', encoding='utf-8') as file:
        return json.load(file)

In [20]:
userId = 'musiclove529'
options = webdriver.ChromeOptions()
options.add_argument('--headleass')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(1)
url = "https://www.acmicpc.net/status?user_id={userId}&result_id=4".format(userId=userId)
sbDict = []
cnt = 1
driver.get(url)
while True:
    table = driver.find_element_by_tag_name('tbody')
    tr = table.find_elements_by_tag_name('tr')
    for i in tr:
        td = i.find_elements_by_tag_name('td')
        sbNum = td[0].text
        uId = td[1].text
        pId = td[2].text
        result = td[3].text
        memory = td[4].text
        time = td[5].text
        lang = td[6].text
        codeLen = td[7].text
        sbTime = td[8].find_element_by_tag_name('a').get_attribute('data-timestamp')
        sbDict.append({'sbNum':sbNum, 'uId':uId, 'pId':pId, 'result':result, 'memory':memory, 'time':time, 'lang':lang, 'codeLen':codeLen, 'sbTime':sbTime})
    try:
        nextPage = driver.find_element_by_id('next_page').get_attribute('href')
    except Exception:
        break
    driver.get(nextPage)
    print(cnt)
    cnt = cnt + 1
try:
    if not os.path.exists('submits/'):
        os.makedirs('submits/')
except OSError:
    print('Error : mkdir')
toJson('submits/' + userId + ".json", sbDict)
driver.quit()

1
2
3
4
5
6
7
8
9
10


In [46]:
categoryDF = pd.read_csv('category.csv', index_col=0, dtype=str, encoding='utf-8')
categoryDF

,pId,category
0,1002,4S
1,1003,19S
2,1004,12S
3,1005,44G
4,1007,31G
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [21]:
# 각 사용자 데이터에서 제출번호, 아이디, 문제 데이터만 추출하여 사용자가 푼 문제에 해당하는 태그 붙이기
file = 'submits/' + userId + '.json'
userDF = pd.read_json(file, dtype = str, encoding='utf-8')
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
215,15501891,musiclove529,1000,맞았습니다!!,14248,108,Java 8,218,1570197725
216,15253710,musiclove529,10172,맞았습니다!!,12628,68,Java 8,254,1569079403
217,15253637,musiclove529,10171,맞았습니다!!,12708,76,Java 8,206,1569079083
218,15253362,musiclove529,10718,맞았습니다!!,12756,76,Java 8,173,1569078144


In [22]:
userDF = userDF[userDF['result']=='맞았습니다!!']
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
215,15501891,musiclove529,1000,맞았습니다!!,14248,108,Java 8,218,1570197725
216,15253710,musiclove529,10172,맞았습니다!!,12628,68,Java 8,254,1569079403
217,15253637,musiclove529,10171,맞았습니다!!,12708,76,Java 8,206,1569079083
218,15253362,musiclove529,10718,맞았습니다!!,12756,76,Java 8,173,1569078144


In [23]:
userDF = userDF[userDF['pId'].isin(categoryDF['pId'])]
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
205,15900766,musiclove529,2753,맞았습니다!!,14252,104,Java 8,285,1572436381
206,15502610,musiclove529,9498,맞았습니다!!,14276,104,Java 8,460,1570200172
207,15502482,musiclove529,1330,맞았습니다!!,14244,104,Java 8,324,1570199763
208,15502384,musiclove529,2588,맞았습니다!!,14240,116,Java 8,368,1570199439


In [24]:
userDF = userDF.sort_values(by='sbNum')
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
211,15502158,musiclove529,1008,맞았습니다!!,14500,108,Java 8,233,1570198629
208,15502384,musiclove529,2588,맞았습니다!!,14240,116,Java 8,368,1570199439
207,15502482,musiclove529,1330,맞았습니다!!,14244,104,Java 8,324,1570199763
206,15502610,musiclove529,9498,맞았습니다!!,14276,104,Java 8,460,1570200172
205,15900766,musiclove529,2753,맞았습니다!!,14252,104,Java 8,285,1572436381
...,...,...,...,...,...,...,...,...,...
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885


In [25]:
if len(userDF) == 0: print()
sbTimes = []
userDF = userDF[['sbTime','uId','pId']]
df = pd.merge(userDF,categoryDF,how='left',on='pId')

# 라벨 붙이기
df['label'] = [1 for i in range(len(df))]

# '라벨', '아이디', '문제', '제출시간', '카테고리'순으로 열 바꾸기
df = df[['label', 'uId', 'pId', 'sbTime', 'category']]
df

,label,uId,pId,sbTime,category
0,1,musiclove529,1008,1570198629,41B
1,1,musiclove529,2588,1570199439,4B
2,1,musiclove529,1330,1570199763,41B
3,1,musiclove529,9498,1570200172,2B
4,1,musiclove529,2753,1572436381,55B
...,...,...,...,...,...
174,1,musiclove529,1546,1649852630,4B
175,1,musiclove529,11659,1650626871,16S
176,1,musiclove529,11660,1650629701,19S
177,1,musiclove529,1253,1652438885,40G


In [29]:
pHis = df['pId']
pStr = ""
for p in pHis:
    pStr = pStr + p + ","
pStr = pStr[:-1]
pStr

'1008,2588,1330,9498,2753,2884,10817,2739,10950,8393,2741,2742,11021,11022,2438,2439,10871,10952,10951,1110,10818,2562,2577,2743,5347,9656,5565,3052,1546,8958,2920,2440,2441,2442,2443,2444,17247,2445,10953,15596,11720,5597,4153,10569,10039,5543,10101,2480,11943,2525,5575,10707,5063,5596,2455,10797,5576,10817,2750,11650,5618,2858,5533,11004,11721,10250,10250,2309,2667,1012,2178,1075,2506,1920,15552,1920,17520,17520,2839,10814,10989,1188,11718,11718,11719,15719,3034,3047,2869,11367,2490,1427,15780,15725,10808,11131,1924,14655,2822,11109,11441,10809,12354,1181,10825,11651,15803,11772,10539,2751,1212,2108,1476,7326,10709,2563,2676,10872,10870,2675,4673,1316,1152,1157,2908,2675,1085,2231,2292,2292,2775,1978,2798,2798,1931,1931,1931,14500,2960,1475,1259,11050,10828,10845,1018,18352,1260,2012,11399,2693,7576,2805,10815,2512,1463,1149,1003,9095,1654,2606,1436,10866,1966,1966,2609,2252,1717,1806,11659,1644,2023,1697,1010,11720,1546,11659,11660,1253,1874'

In [30]:
cHis = df['category']
cStr = ""
for c in cHis:
    cStr = cStr + c + ","
cStr = cStr[:-1]
cStr

'41B,4B,41B,2B,55B,4B,2B,41B,41B,55B,2B,2B,41B,41B,2B,2B,55B,41B,41B,55B,55B,2B,41B,51B,65S,12S,41B,4B,4B,51B,2B,2B,2B,2B,2B,2B,4B,2B,7B,41B,7B,2B,4B,4B,4B,4B,2B,4B,55B,4B,41B,4B,4B,41B,41B,2B,54B,2B,54B,15S,31B,31B,41B,15S,51B,41B,41B,46B,24S,24S,1S,4B,41B,26S,41B,26S,4B,4B,48B,15S,15S,39G,51B,51B,51B,55B,4B,2B,4B,51B,2B,3S,4B,7S,51B,41B,2B,33B,15S,4B,16S,51B,11S,3S,15S,15S,4S,41B,4B,15S,32B,55S,31S,55B,2S,2B,4S,55B,55B,51B,55S,51S,51B,51B,55B,51B,4B,11B,4B,4B,55B,22S,11B,11B,15S,15S,15S,11G,22S,2S,51B,52B,13S,57S,11S,1S,24S,15S,15S,15S,1G,20S,20S,20S,19S,19S,19S,19S,20S,24S,11S,17S,57S,57S,65S,14G,0G,12G,16S,22G,50G,1S,48S,7B,4B,16S,19S,40G,13S'

In [31]:
tHis = df['sbTime']
tStr = ""
for t in tHis:
    tStr = tStr + t + ","
tStr = tStr[:-1]
tStr

'1570198629,1570199439,1570199763,1570200172,1572436381,1572437660,1572438312,1572511898,1572520733,1572521143,1572848863,1572849016,1572881440,1572881604,1572881833,1572882144,1572882820,1578639832,1578755044,1578756765,1580361768,1580362380,1580383031,1580384026,1580386296,1580390106,1580390775,1580526621,1581424311,1581426841,1581428592,1581489376,1581489994,1581490716,1581491308,1581503056,1581504920,1581777801,1581780896,1581781417,1583828728,1583829774,1583834951,1583836877,1583877736,1583878441,1583879176,1583880228,1583889451,1583892652,1583896450,1583899933,1583900411,1583901063,1583932599,1585065809,1586597011,1587189772,1587190835,1587192045,1587489631,1587493792,1587495452,1587499449,1587500648,1587633193,1587634578,1587643535,1589283422,1589285992,1590666617,1598946232,1598978974,1598981732,1600760048,1601969458,1602243232,1602243822,1602334768,1602338229,1602340129,1602349441,1602354569,1602354594,1602354679,1602764398,1604251461,1604253031,1604254215,1605787126,160578819

In [69]:
test_problem = categoryDF.copy()
test_problem

,pId,category
0,1002,4S
1,1003,19S
2,1004,12S
3,1005,44G
4,1007,31G
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [70]:
solved_problem = df['pId']
solved_problem

0       1008
1       2588
2       1330
3       9498
4       2753
       ...  
174     1546
175    11659
176    11660
177     1253
178     1874
Name: pId, Length: 179, dtype: object

In [71]:
remove_index = test_problem[test_problem['pId'].isin(solved_problem)].index
remove_index

Int64Index([   1,    5,    7,    9,   14,   56,   63,   82,  105,  107,
            ...
            5677, 6095, 6122, 6188, 6194, 6226, 6237, 6941, 7082, 7433],
           dtype='int64', length=165)

In [72]:
test_problem = test_problem.drop(remove_index)
test_problem

,pId,category
0,1002,4S
2,1004,12S
3,1005,44G
4,1007,31G
6,1009,55B
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [68]:
now = int(datetime.now().timestamp())
now

1652816212

In [75]:
index = test_problem.index
label = "0"
uId = "musiclove529"
now = str(int(datetime.now().timestamp()))
f = open('test_'+ uId,'w')
for i in index:
    data = test_problem.loc[i]
    pId = data['pId']
    cate = data['category']
    f.write(label + '\t' + uId + '\t' + pId + '\t' + cate + '\t' + now + '\t' + pStr + '\t' + cStr + '\t' + tStr + '\n')


In [17]:
import pandas as pd

In [18]:
problems = pd.read_csv("D:/AlgorithmProblemRecommender/Recommenders/recommend/problems.csv",index_col=0)
problems

,Mathematics,Implementation,DynamicProgramming,GraphTheory,DataStructures,String,Greedy,Bruteforcing,GraphTraversal,Sorting,...,Hashing,TopologicalSorting,Floyd_warshall,SlidingWindow,Deque,LongestIncreasingSequenceInO(nLogN),Queue,0_1Bfs,TravellingSalesmanProblem,level
1002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1003,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
1005,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,13
1007,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24936,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6
24937,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
24938,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,7
24939,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [19]:
tags = pd.read_json('D:/AlgorithmProblemRecommender/Recommenders/recommend/tagData.json')
tags

,name,engName,url,cnt
0,수학,Mathematics,https://www.acmicpc.net/problem/tag/124,3807
1,구현,Implementation,https://www.acmicpc.net/problem/tag/102,3403
2,다이나믹 프로그래밍,Dynamic Programming,https://www.acmicpc.net/problem/tag/25,2375
3,그래프 이론,Graph Theory,https://www.acmicpc.net/problem/tag/7,2222
4,자료 구조,Data Structures,https://www.acmicpc.net/problem/tag/175,2185
5,문자열,String,https://www.acmicpc.net/problem/tag/158,1520
6,그리디 알고리즘,Greedy,https://www.acmicpc.net/problem/tag/33,1320
7,브루트포스 알고리즘,Bruteforcing,https://www.acmicpc.net/problem/tag/125,1280
8,그래프 탐색,Graph Traversal,https://www.acmicpc.net/problem/tag/11,1211
9,정렬,Sorting,https://www.acmicpc.net/problem/tag/97,1030


In [20]:
tagNames = []
for i in range(len(tags)):
    engName = tags.loc[i]['engName']
    tagName = engName.replace(" ", "").replace("/", "").replace("-","_").replace("*","asterisk")
    tagNames.append(tagName)
tagNames

['Mathematics',
 'Implementation',
 'DynamicProgramming',
 'GraphTheory',
 'DataStructures',
 'String',
 'Greedy',
 'Bruteforcing',
 'GraphTraversal',
 'Sorting',
 'Geometry',
 'NumberTheory',
 'Tree',
 'SegmentTree',
 'BinarySearch',
 'Breadth_firstSearch',
 'Arithmetic',
 'Simulation',
 'Depth_firstSearch',
 'Combinatorics',
 'Ad_hoc',
 'PrefixSum',
 'Constructive',
 'Bitmask',
 "Dijkstra's",
 'Backtracking',
 'DisjointSet',
 'DivideAndConquer',
 'SetMapByHashing',
 'SetMapByTrees',
 'PriorityQueue',
 'Stack',
 'DynamicProgrammingOnTrees',
 'Two_pointer',
 'PrimalityTest',
 'GameTheory',
 'ExponentiationBySquaring',
 'Recursion',
 'ParametricSearch',
 'SieveOfEratosthenes',
 'ProbabilityTheory',
 'Knapsack',
 'MinimumSpanningTree',
 'EuclideanAlgorithm',
 'ValueCoordinateCompression',
 'Hashing',
 'TopologicalSorting',
 'Floyd–warshall',
 'SlidingWindow',
 'Deque',
 'LongestIncreasingSequenceInO(nLogN)',
 'Queue',
 '0_1Bfs',
 'TravellingSalesmanProblem']

In [21]:
tags['tagName'] = tagNames
tags

,name,engName,url,cnt,tagName
0,수학,Mathematics,https://www.acmicpc.net/problem/tag/124,3807,Mathematics
1,구현,Implementation,https://www.acmicpc.net/problem/tag/102,3403,Implementation
2,다이나믹 프로그래밍,Dynamic Programming,https://www.acmicpc.net/problem/tag/25,2375,DynamicProgramming
3,그래프 이론,Graph Theory,https://www.acmicpc.net/problem/tag/7,2222,GraphTheory
4,자료 구조,Data Structures,https://www.acmicpc.net/problem/tag/175,2185,DataStructures
5,문자열,String,https://www.acmicpc.net/problem/tag/158,1520,String
6,그리디 알고리즘,Greedy,https://www.acmicpc.net/problem/tag/33,1320,Greedy
7,브루트포스 알고리즘,Bruteforcing,https://www.acmicpc.net/problem/tag/125,1280,Bruteforcing
8,그래프 탐색,Graph Traversal,https://www.acmicpc.net/problem/tag/11,1211,GraphTraversal
9,정렬,Sorting,https://www.acmicpc.net/problem/tag/97,1030,Sorting


In [22]:
tags = tags[['name','engName','tagName','url']]
tags

,name,engName,tagName,url
0,수학,Mathematics,Mathematics,https://www.acmicpc.net/problem/tag/124
1,구현,Implementation,Implementation,https://www.acmicpc.net/problem/tag/102
2,다이나믹 프로그래밍,Dynamic Programming,DynamicProgramming,https://www.acmicpc.net/problem/tag/25
3,그래프 이론,Graph Theory,GraphTheory,https://www.acmicpc.net/problem/tag/7
4,자료 구조,Data Structures,DataStructures,https://www.acmicpc.net/problem/tag/175
5,문자열,String,String,https://www.acmicpc.net/problem/tag/158
6,그리디 알고리즘,Greedy,Greedy,https://www.acmicpc.net/problem/tag/33
7,브루트포스 알고리즘,Bruteforcing,Bruteforcing,https://www.acmicpc.net/problem/tag/125
8,그래프 탐색,Graph Traversal,GraphTraversal,https://www.acmicpc.net/problem/tag/11
9,정렬,Sorting,Sorting,https://www.acmicpc.net/problem/tag/97


In [7]:
tags.to_csv('tags.csv')

In [23]:
index = problems.index
index

Int64Index([ 1002,  1003,  1004,  1005,  1007,  1008,  1009,  1010,  1011,
             1012,
            ...
            24928, 24929, 24930, 24933, 24935, 24936, 24937, 24938, 24939,
            24940],
           dtype='int64', length=9573)

In [24]:
col = problems.columns
col

Index(['Mathematics', 'Implementation', 'DynamicProgramming', 'GraphTheory',
       'DataStructures', 'String', 'Greedy', 'Bruteforcing', 'GraphTraversal',
       'Sorting', 'Geometry', 'NumberTheory', 'Tree', 'SegmentTree',
       'BinarySearch', 'Breadth_firstSearch', 'Arithmetic', 'Simulation',
       'Depth_firstSearch', 'Combinatorics', 'Ad_hoc', 'PrefixSum',
       'Constructive', 'Bitmask', 'Dijkstra's', 'Backtracking', 'DisjointSet',
       'DivideAndConquer', 'SetMapByHashing', 'SetMapByTrees', 'PriorityQueue',
       'Stack', 'DynamicProgrammingOnTrees', 'Two_pointer', 'PrimalityTest',
       'GameTheory', 'ExponentiationBySquaring', 'Recursion',
       'ParametricSearch', 'SieveOfEratosthenes', 'ProbabilityTheory',
       'Knapsack', 'MinimumSpanningTree', 'EuclideanAlgorithm',
       'ValueCoordinateCompression', 'Hashing', 'TopologicalSorting',
       'Floyd_warshall', 'SlidingWindow', 'Deque',
       'LongestIncreasingSequenceInO(nLogN)', 'Queue', '0_1Bfs',
       'Travel

In [25]:
tagNames = col[:-1]
tagNames

Index(['Mathematics', 'Implementation', 'DynamicProgramming', 'GraphTheory',
       'DataStructures', 'String', 'Greedy', 'Bruteforcing', 'GraphTraversal',
       'Sorting', 'Geometry', 'NumberTheory', 'Tree', 'SegmentTree',
       'BinarySearch', 'Breadth_firstSearch', 'Arithmetic', 'Simulation',
       'Depth_firstSearch', 'Combinatorics', 'Ad_hoc', 'PrefixSum',
       'Constructive', 'Bitmask', 'Dijkstra's', 'Backtracking', 'DisjointSet',
       'DivideAndConquer', 'SetMapByHashing', 'SetMapByTrees', 'PriorityQueue',
       'Stack', 'DynamicProgrammingOnTrees', 'Two_pointer', 'PrimalityTest',
       'GameTheory', 'ExponentiationBySquaring', 'Recursion',
       'ParametricSearch', 'SieveOfEratosthenes', 'ProbabilityTheory',
       'Knapsack', 'MinimumSpanningTree', 'EuclideanAlgorithm',
       'ValueCoordinateCompression', 'Hashing', 'TopologicalSorting',
       'Floyd_warshall', 'SlidingWindow', 'Deque',
       'LongestIncreasingSequenceInO(nLogN)', 'Queue', '0_1Bfs',
       'Travel

In [ ]:
problemInfo = pd.DataFrame(columns=['id', 'title', 'url','successCnt', 'submissionCnt', 'answerRate', 'level'])
for t in tags:
    data = loadJson('D:/AlgorithmProblemRecommender/Recommenders/recommend/problems/{}.json'.format(t))
    dataDF = pd.DataFrame(data)
    problems = pd.concat([problems, dataDF])
problemInfo

In [ ]:
problemInfo.drop_duplicates(subset='id', inplace=True, ignore_index=True)
problemInfo

In [ ]:
problemInfo.rename(columns={'id':'pId'}, inplace=True)
problemInfo.sort_values('level', ascending=False, inplace=True, ignore_index=True)
problemInfo

In [ ]:
problems.to_csv("problemInfo.csv")

In [28]:
problemInfo = pd.read_csv("problemInfo.csv", index_col=0)
problemInfo

,pId,title,url,successCnt,submissionCnt,answerRate,level
0,16156,장애물 달리기,https://www.acmicpc.net/problem/16156,69,259,40.116%,17
1,16399,드라이브,https://www.acmicpc.net/problem/16399,87,366,30.420%,17
2,10729,업적의 노예 1,https://www.acmicpc.net/problem/10729,18,47,51.429%,17
3,21036,Mini Market,https://www.acmicpc.net/problem/21036,8,54,22.222%,17
4,10335,There is No Alternative,https://www.acmicpc.net/problem/10335,44,111,44.000%,17
...,...,...,...,...,...,...,...
9568,20352,Circus,https://www.acmicpc.net/problem/20352,392,555,85.590%,2
9569,20215,Cutting Corners,https://www.acmicpc.net/problem/20215,391,477,87.082%,2
9570,22015,金平糖 (Konpeito),https://www.acmicpc.net/problem/22015,387,519,84.498%,2
9571,17863,FYI,https://www.acmicpc.net/problem/17863,958,1259,80.912%,2


In [49]:
problemInfo.index = list(problemInfo['pId'])
problemInfo

,pId,title,url,successCnt,submissionCnt,answerRate,level
16156,16156,장애물 달리기,https://www.acmicpc.net/problem/16156,69,259,40.116%,17
16399,16399,드라이브,https://www.acmicpc.net/problem/16399,87,366,30.420%,17
10729,10729,업적의 노예 1,https://www.acmicpc.net/problem/10729,18,47,51.429%,17
21036,21036,Mini Market,https://www.acmicpc.net/problem/21036,8,54,22.222%,17
10335,10335,There is No Alternative,https://www.acmicpc.net/problem/10335,44,111,44.000%,17
...,...,...,...,...,...,...,...
20352,20352,Circus,https://www.acmicpc.net/problem/20352,392,555,85.590%,2
20215,20215,Cutting Corners,https://www.acmicpc.net/problem/20215,391,477,87.082%,2
22015,22015,金平糖 (Konpeito),https://www.acmicpc.net/problem/22015,387,519,84.498%,2
17863,17863,FYI,https://www.acmicpc.net/problem/17863,958,1259,80.912%,2


In [52]:
problemInfo = problemInfo[['title', 'successCnt']]
problemInfo

,title,successCnt
16156,장애물 달리기,69
16399,드라이브,87
10729,업적의 노예 1,18
21036,Mini Market,8
10335,There is No Alternative,44
...,...,...
20352,Circus,392
20215,Cutting Corners,391
22015,金平糖 (Konpeito),387
17863,FYI,958


In [56]:
problemInfo.loc[1002]

title            터렛
successCnt    25221
Name: 1002, dtype: object

In [65]:
problemInfo.dtypes

title         object
successCnt     int64
dtype: object

In [66]:
problemDict = []
for i in index:
    data = problems.loc[i]
    info = problemInfo.loc[i]
    title = info['title']
    successCnt = int(info['successCnt'])
    tagDict = []
    for tag in tagNames:
        if(data[tag]):
            for j in range(len(tags)):
                tagData = tags.loc[j]
                if tag == tagData['tagName']:
                    tagDict.append({'tagName':tagData['name'], 'engName':tagData['engName'], 'url':tagData['url']})
    levelNum = data['level']
    if levelNum <= 5:
        level = 'Bronze'+str(6-levelNum)
    elif levelNum <= 10:
        level = 'Silver'+str(11-levelNum)
    elif levelNum <= 15:
        level = 'Gold'+str(16-levelNum)
    else:
        level = 'Platinum'+str(21-levelNum)
    problemDict.append({'pId':i, 'title':title, 'successCnt':successCnt, 'tags':tagDict, 'level':level, 'url':'https://www.acmicpc.net/problem/{}'.format(i)})
problemDict

[{'pId': 1002,
  'title': '터렛',
  'successCnt': 25221,
  'tags': [{'tagName': '수학',
    'engName': 'Mathematics',
    'url': 'https://www.acmicpc.net/problem/tag/124'},
   {'tagName': '기하학',
    'engName': 'Geometry',
    'url': 'https://www.acmicpc.net/problem/tag/100'}],
  'level': 'Silver4',
  'url': 'https://www.acmicpc.net/problem/1002'},
 {'pId': 1003,
  'title': '피보나치 함수',
  'successCnt': 34025,
  'tags': [{'tagName': '다이나믹 프로그래밍',
    'engName': 'Dynamic Programming',
    'url': 'https://www.acmicpc.net/problem/tag/25'}],
  'level': 'Silver3',
  'url': 'https://www.acmicpc.net/problem/1003'},
 {'pId': 1004,
  'title': '어린 왕자',
  'successCnt': 8337,
  'tags': [{'tagName': '기하학',
    'engName': 'Geometry',
    'url': 'https://www.acmicpc.net/problem/tag/100'}],
  'level': 'Silver3',
  'url': 'https://www.acmicpc.net/problem/1004'},
 {'pId': 1005,
  'title': 'ACM Craft',
  'successCnt': 8028,
  'tags': [{'tagName': '다이나믹 프로그래밍',
    'engName': 'Dynamic Programming',
    'url': 'ht

In [58]:
import json

def toJson(dir, dict):
    with open(dir, 'w', encoding='utf-8') as file:
        json.dump(dict, file, ensure_ascii=False, indent='\t')

def loadJson(dir):
    with open(dir, 'r', encoding='utf-8') as file:
        return json.load(file)

In [67]:
toJson('problemData.json', problemDict)

In [1]:
import pandas as pd

In [2]:
dir = 'D:/AlgorithmProblemRecommender/Recommenders/recommend/'
categoryDF = pd.read_csv(dir + 'category.csv', index_col=0, dtype=str, encoding='utf-8')
userId = "musiclove529"
file = dir + 'data/{userId}/{userId}.json'.format(userId=userId)
userDF = pd.read_json(file, dtype = str, encoding='utf-8')
# 맞은 문제만 추출
userDF = userDF[userDF['result']=='맞았습니다!!']
# 문제셋에 해당하는 문제만 추출
userDF = userDF[userDF['pId'].isin(categoryDF['pId'])]
userDF = userDF[['pId']]
# pId dtype int형으로 변환
userDF['pId'] = pd.to_numeric(userDF['pId'])
userDF

,pId
0,4949
1,10026
2,2164
3,1874
4,1253
...,...
208,2753
209,9498
210,1330
211,2588


In [3]:
problems = pd.read_csv(dir+"problems.csv")
col = ['pId'] + list(problems.columns)[1:]
problems.columns = col
problems

,pId,Mathematics,Implementation,DynamicProgramming,GraphTheory,DataStructures,String,Greedy,Bruteforcing,GraphTraversal,...,Hashing,TopologicalSorting,Floyd_warshall,SlidingWindow,Deque,LongestIncreasingSequenceInO(nLogN),Queue,0_1Bfs,TravellingSalesmanProblem,level
0,1002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,1003,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,1004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,1005,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,13
4,1007,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9568,24936,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
9569,24937,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
9570,24938,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,7
9571,24939,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [4]:
df = pd.merge(userDF, problems, how='left', on='pId')
df

,pId,Mathematics,Implementation,DynamicProgramming,GraphTheory,DataStructures,String,Greedy,Bruteforcing,GraphTraversal,...,Hashing,TopologicalSorting,Floyd_warshall,SlidingWindow,Deque,LongestIncreasingSequenceInO(nLogN),Queue,0_1Bfs,TravellingSalesmanProblem,level
0,4949,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,10026,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,11
2,2164,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,7
3,1874,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,1253,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2753,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
178,9498,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
179,1330,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
180,2588,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [51]:
df.sort_values(by='level', ascending=False, inplace=True)
df

,pId,Mathematics,Implementation,DynamicProgramming,GraphTheory,DataStructures,String,Greedy,Bruteforcing,GraphTraversal,...,Hashing,TopologicalSorting,Floyd_warshall,SlidingWindow,Deque,LongestIncreasingSequenceInO(nLogN),Queue,0_1Bfs,TravellingSalesmanProblem,level
12,1644,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
16,2252,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,13
4,1253,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
15,1717,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
14,1806,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2753,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
179,1330,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
180,2588,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
136,5543,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [101]:
tags = df.columns[1:-1]
tags

Index(['Mathematics', 'Implementation', 'DynamicProgramming', 'GraphTheory',
       'DataStructures', 'String', 'Greedy', 'Bruteforcing', 'GraphTraversal',
       'Sorting', 'Geometry', 'NumberTheory', 'Tree', 'SegmentTree',
       'BinarySearch', 'Breadth_firstSearch', 'Arithmetic', 'Simulation',
       'Depth_firstSearch', 'Combinatorics', 'Ad_hoc', 'PrefixSum',
       'Constructive', 'Bitmask', 'Dijkstra's', 'Backtracking', 'DisjointSet',
       'DivideAndConquer', 'SetMapByHashing', 'SetMapByTrees', 'PriorityQueue',
       'Stack', 'DynamicProgrammingOnTrees', 'Two_pointer', 'PrimalityTest',
       'GameTheory', 'ExponentiationBySquaring', 'Recursion',
       'ParametricSearch', 'SieveOfEratosthenes', 'ProbabilityTheory',
       'Knapsack', 'MinimumSpanningTree', 'EuclideanAlgorithm',
       'ValueCoordinateCompression', 'Hashing', 'TopologicalSorting',
       'Floyd_warshall', 'SlidingWindow', 'Deque',
       'LongestIncreasingSequenceInO(nLogN)', 'Queue', '0_1Bfs',
       'Travel

In [98]:
strong = df.head(100)
strong

,pId,Mathematics,Implementation,DynamicProgramming,GraphTheory,DataStructures,String,Greedy,Bruteforcing,GraphTraversal,...,Hashing,TopologicalSorting,Floyd_warshall,SlidingWindow,Deque,LongestIncreasingSequenceInO(nLogN),Queue,0_1Bfs,TravellingSalesmanProblem,level
12,1644,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
16,2252,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,13
4,1253,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
15,1717,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
14,1806,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,15552,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
52,2292,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
87,10808,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
94,3047,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [99]:
strongCountDict = {}
for t in tags:
    if 1 in strong[t].value_counts().keys():
        cnt = strong[t].value_counts()[1]
    else: cnt = 0
    strongCountDict[t] = cnt
strongCountDict

{'Mathematics': 30,
 'Implementation': 26,
 'DynamicProgramming': 8,
 'GraphTheory': 10,
 'DataStructures': 12,
 'String': 11,
 'Greedy': 7,
 'Bruteforcing': 6,
 'GraphTraversal': 9,
 'Sorting': 20,
 'Geometry': 1,
 'NumberTheory': 8,
 'Tree': 0,
 'SegmentTree': 0,
 'BinarySearch': 7,
 'Breadth_firstSearch': 9,
 'Arithmetic': 6,
 'Simulation': 3,
 'Depth_firstSearch': 5,
 'Combinatorics': 2,
 'Ad_hoc': 0,
 'PrefixSum': 4,
 'Constructive': 0,
 'Bitmask': 0,
 "Dijkstra's": 1,
 'Backtracking': 1,
 'DisjointSet': 1,
 'DivideAndConquer': 0,
 'SetMapByHashing': 1,
 'SetMapByTrees': 2,
 'PriorityQueue': 0,
 'Stack': 3,
 'DynamicProgrammingOnTrees': 0,
 'Two_pointer': 3,
 'PrimalityTest': 4,
 'GameTheory': 1,
 'ExponentiationBySquaring': 0,
 'Recursion': 0,
 'ParametricSearch': 3,
 'SieveOfEratosthenes': 3,
 'ProbabilityTheory': 0,
 'Knapsack': 0,
 'MinimumSpanningTree': 0,
 'EuclideanAlgorithm': 2,
 'ValueCoordinateCompression': 0,
 'Hashing': 0,
 'TopologicalSorting': 1,
 'Floyd_warshall': 0

In [97]:
sortedStrongCountDict = dict(sorted(strongCountDict.items(), key=lambda x: x[1], reverse=True))
sortedStrongCountDict

{'Mathematics': 51,
 'Implementation': 48,
 'String': 23,
 'Sorting': 22,
 'Arithmetic': 18,
 'NumberTheory': 8,
 'Greedy': 6,
 'Bruteforcing': 5,
 'DataStructures': 4,
 'Geometry': 4,
 'PrimalityTest': 4,
 'Simulation': 3,
 'SieveOfEratosthenes': 3,
 'DynamicProgramming': 2,
 'BinarySearch': 2,
 'Combinatorics': 2,
 'Two_pointer': 2,
 'EuclideanAlgorithm': 2,
 'Queue': 2,
 'Backtracking': 1,
 'SetMapByHashing': 1,
 'Stack': 1,
 'GraphTheory': 0,
 'GraphTraversal': 0,
 'Tree': 0,
 'SegmentTree': 0,
 'Breadth_firstSearch': 0,
 'Depth_firstSearch': 0,
 'Ad_hoc': 0,
 'PrefixSum': 0,
 'Constructive': 0,
 'Bitmask': 0,
 "Dijkstra's": 0,
 'DisjointSet': 0,
 'DivideAndConquer': 0,
 'SetMapByTrees': 0,
 'PriorityQueue': 0,
 'DynamicProgrammingOnTrees': 0,
 'GameTheory': 0,
 'ExponentiationBySquaring': 0,
 'Recursion': 0,
 'ParametricSearch': 0,
 'ProbabilityTheory': 0,
 'Knapsack': 0,
 'MinimumSpanningTree': 0,
 'ValueCoordinateCompression': 0,
 'Hashing': 0,
 'TopologicalSorting': 0,
 'Floyd_

In [100]:
list(strongCountDict.keys())[:5]

['Mathematics',
 'Implementation',
 'DynamicProgramming',
 'GraphTheory',
 'DataStructures']

In [1]:
import pandas as pd

In [15]:
userId = "musiclove529"
# 문제 태그 데이터 가져오기
dir = 'D:/AlgorithmProblemRecommender/Recommenders/recommend/'
categoryDF = pd.read_csv(dir + 'category.csv', index_col=0, dtype=str, encoding='utf-8')
file = dir + 'data/{userId}/{userId}.json'.format(userId=userId)
userDF = pd.read_json(file, dtype = str, encoding='utf-8')
# 맞은 문제만 추출
userDF = userDF[userDF['result']=='맞았습니다!!']
# 문제셋에 해당하는 문제만 추출
userDF = userDF[userDF['pId'].isin(categoryDF['pId'])]
# 중복 제거
userDF.drop_duplicates(subset=['pId'], keep='last', inplace=True, ignore_index=True)
# 문제 제출 번호에 따라 오름차순으로 정렬
userDF = userDF.sort_values(by='sbNum')
if len(userDF) == 0: print()
# 사용자 데이터에서 제출시간, 아이디, 문제 데이터만 추출하여 사용자가 푼 문제에 해당하는 태그 붙이기
userDF = userDF[['sbTime','uId','pId']]
df = pd.merge(userDF,categoryDF,how='left',on='pId')
df

,sbTime,uId,pId,category
0,1570198629,musiclove529,1008,41B
1,1570199439,musiclove529,2588,4B
2,1570199763,musiclove529,1330,41B
3,1570200172,musiclove529,9498,2B
4,1572436381,musiclove529,2753,55B
...,...,...,...,...
163,1652438885,musiclove529,1253,40G
164,1652449450,musiclove529,1874,13S
165,1653478075,musiclove529,2164,57S
166,1653501526,musiclove529,10026,24G


In [16]:
df[df.duplicated()]

,sbTime,uId,pId,category
